In [1]:
# Let's beggin by downloading the data from SGS API

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_sgs(series_id, daily=False):
    base_url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{series_id}/dados?formato=json"

    # If it is a daily series → need to pass dataInicial
    if daily:
        start_date = (datetime.today() - timedelta(days=3650)).strftime("%d/%m/%Y")
        url = base_url + f"&dataInicial={start_date}"
    else:
        url = base_url

    r = requests.get(url)
    data = r.json()

    if isinstance(data, dict):
        print(f"⚠️ ERRO na série {series_id}: {data}")
        return pd.DataFrame()

    df = pd.DataFrame(data)
    df['valor'] = df['valor'].astype(float)
    df['data'] = pd.to_datetime(df['data'], dayfirst=True)
    df = df.sort_values('data')
    return df

series_dict = {
    "selic_meta": {"id": 4189, "daily": False},
    "selic_efetiva": {"id": 4390, "daily": False},
    "ipca": {"id": 433, "daily": False},
    "ibc_br": {"id": 24363, "daily": False},
    "usd_brl": {"id": 10813, "daily": True},
    "unemployment": {"id": 24369, "daily": False},
    "debt_dbgg": {"id": 4504, "daily": False}
}

raw_data = {}

for name, info in series_dict.items():
    print(f"⬇️ Baixando {name} ({info['id']})...")
    df = get_sgs(info["id"], daily=info["daily"])

    if not df.empty:
        print(f"{name}: {df.shape[0]} linhas baixadas.")
    else:
        print(f"⚠️ {name}: NENHUM dado baixado.")

    df.to_csv(f"../data/raw/{name}.csv", index=False)

    raw_data[name] = df


⬇️ Baixando selic_meta (4189)...
selic_meta: 472 linhas baixadas.
⬇️ Baixando selic_efetiva (4390)...
selic_efetiva: 472 linhas baixadas.
⬇️ Baixando ipca (433)...
ipca: 549 linhas baixadas.
⬇️ Baixando ibc_br (24363)...
ibc_br: 273 linhas baixadas.
⬇️ Baixando usd_brl (10813)...
usd_brl: 2509 linhas baixadas.
⬇️ Baixando unemployment (24369)...
unemployment: 163 linhas baixadas.
⬇️ Baixando debt_dbgg (4504)...
debt_dbgg: 285 linhas baixadas.
